https://apps.skillfactory.ru/learning/course/course-v1:SkillFactory+DSPR-2.0+14JULY2021/block-v1:SkillFactory+DSPR-2.0+14JULY2021+type@sequential+block@8bd0c0f84e0844d4a884df20d4a6d9f1/block-v1:SkillFactory+DSPR-2.0+14JULY2021+type@vertical+block@17fff64d0efd460aae473417c2440c47

covid_data — данные о показателях заболеваемости за каждый день в различных регионах 187 стран
Мы будем работать со следующими столбцами:
date — дата наблюдения;
province/state — наименование провинции/штата;
country — наименование страны;
confirmed — общее число зафиксированных случаев на указанный день;
deaths — общее число зафиксированных смертей на указанный день;
recovered — общее число выздоровлений на указанный день.


country_vaccinations — данные о процессе вакцинирования людей в 221 стране мира.

Данная таблица содержит следующие столбцы:

country — наименование страны;
date — дата наблюдения;
total_vaccinations — общее число введённых вакцин в стране на указанный день;
people_vaccinated — общее число привитых первым компонентом в стране на указанный день;
people_vaccinated_per_hundred — процент привитых первым компонентом в стране на указанный день (рассчитывается как );
people_fully_vaccinated — общее число привитых вторым компонентом в стране на указанный день (первый компонент уже был введён им ранее);
people_fully_vaccinated_per_hundred — процент привитых вторым компонентом в стране на указанный день (рассчитывается как );
daily_vaccination — ежедневная вакцинация (число вакцинированных в указанный день);
vaccines — комбинации вакцин, используемые в стране.

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import pandas as pd

In [2]:
import os
os.chdir(r'C:\unique_data\rep_fo_sf\PYTHON_13_data_mining')

print(os.listdir())

covid_data = pd.read_csv('data/covid_data.csv')
display(covid_data.head())

['data', 'DS_2_5_Types_of_visualization.ipynb', 'DS_2_5_Visualization_Pandas.ipynb', 'DS_2_5_Visualization_Plotly.ipynb', 'DS_2_5_Visualization_Seaborn.ipynb', 'merged_data.csv', 'PYTON_13_3_covid_data_country_vaccinations.ipynb', 'PYTON_13_9_Consolidation_of_knowledge.ipynb', 'Диаграмма без названия - Page-1.mhtml']


,date,province/state,country,confirmed,deaths,recovered
0,01/22/2020,Anhui,China,1.0,0.0,0.0
1,01/22/2020,Beijing,China,14.0,0.0,0.0
2,01/22/2020,Chongqing,China,6.0,0.0,0.0
3,01/22/2020,Fujian,China,1.0,0.0,0.0
4,01/22/2020,Gansu,China,0.0,0.0,0.0


In [3]:
vaccinations_data = pd.read_csv('data/country_vaccinations.csv')
display(vaccinations_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42795 entries, 0 to 42794
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   country                              42795 non-null  object 
 1   iso_code                             42795 non-null  object 
 2   date                                 42795 non-null  object 
 3   total_vaccinations                   23457 non-null  float64
 4   people_vaccinated                    22371 non-null  float64
 5   people_fully_vaccinated              19462 non-null  float64
 6   daily_vaccinations_raw               19251 non-null  float64
 7   daily_vaccinations                   42558 non-null  float64
 8   total_vaccinations_per_hundred       23457 non-null  float64
 9   people_vaccinated_per_hundred        22371 non-null  float64
 10  people_fully_vaccinated_per_hundred  19462 non-null  float64
 11  daily_vaccinations_per_milli

None

In [4]:
vaccinations_data = vaccinations_data[['country', 'date', 'total_vaccinations', 'people_vaccinated', 'people_vaccinated_per_hundred','people_fully_vaccinated', 'people_fully_vaccinated_per_hundred',
     'daily_vaccinations', 'vaccines']
]
display(vaccinations_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42795 entries, 0 to 42794
Data columns (total 9 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   country                              42795 non-null  object 
 1   date                                 42795 non-null  object 
 2   total_vaccinations                   23457 non-null  float64
 3   people_vaccinated                    22371 non-null  float64
 4   people_vaccinated_per_hundred        22371 non-null  float64
 5   people_fully_vaccinated              19462 non-null  float64
 6   people_fully_vaccinated_per_hundred  19462 non-null  float64
 7   daily_vaccinations                   42558 non-null  float64
 8   vaccines                             42795 non-null  object 
dtypes: float64(6), object(3)
memory usage: 2.9+ MB


None

ПРЕДОБРАБОТКА ДАННЫХ
Опираясь на замечания выше, выполним небольшую предобработку.
В таблице covid_data:

In [5]:
"""Группируем таблицу по дате и названию страны и рассчитываем суммарные показатели по всем регионам. Тем самым переходим от данных по регионам к данным по странам:"""
covid_data = covid_data.groupby(
    ['date', 'country'], 
    as_index=False
)[['confirmed', 'deaths', 'recovered']].sum()

"""реобразуем даты в формат datetime с помощью функции pd.to_datetime():"""
covid_data['date'] = pd.to_datetime(covid_data['date'])

""""оздадим признак больных на данный момент (active). Для этого вычтем из общего числа зафиксированных случаев число смертей и число выздоровевших пациентов:"""
covid_data['active'] = covid_data['confirmed'] - covid_data['deaths'] - covid_data['recovered']

"""Создадим признак ежедневного прироста числа заболевших, умерших и выздоровевших людей.
Для этого отсортируем данные по названиям стран, а затем по датам. 
После этого произведём группировку по странам и рассчитаем разницу между «вчера и сегодня» с помощью метода diff():"""
covid_data = covid_data.sort_values(by=['country', 'date'])
covid_data['daily_confirmed'] = covid_data.groupby('country')['confirmed'].diff()
covid_data['daily_deaths'] = covid_data.groupby('country')['deaths'].diff()
covid_data['daily_recovered'] = covid_data.groupby('country')['recovered'].diff()

covid_data.head()

,date,country,confirmed,deaths,recovered,active,daily_confirmed,daily_deaths,daily_recovered
11337,2020-02-24,Afghanistan,1.0,0.0,0.0,1.0,NaN,NaN,NaN
11570,2020-02-25,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0
11807,2020-02-26,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0
12051,2020-02-27,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0
12299,2020-02-28,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [6]:
"""Задание 3.1 За какой период представлены данные в таблице covid_data?
В качестве ответа введите даты в формате datetime (без указания времени).
Введите ответ в следующем формате (без пробелов): ГГГГ-ММ-ДД-ГГГГ-ММ-ДД"""
time_start = covid_data["date"].min()
time_finish = covid_data["date"].max()
print(time_start, time_finish)

2020-01-22 00:00:00 2021-05-29 00:00:00


In [7]:
"""Задание 3.2 За какой период представлены данные в таблице vaccinations_data? В качестве ответа введите даты в формате datetime без указания времени.
Введите ответ в следующем формате (без пробелов): ГГГГ-ММ-ДД-ГГГГ-ММ-ДД"""
time_start = vaccinations_data["date"].min()
time_finish = vaccinations_data["date"].max()
print(time_start, time_finish)

2020-12-02 2021-09-06


In [8]:
"""Задание 3.3 
С помощью метода merge() объедините таблицы covid_data и vaccinations_data по столбцам date и country.
Тип объединения выставьте так, чтобы в результирующую таблицу попали только наблюдения за период, вычисленный в задании 3.1.
То есть в результирующую таблицу должны попасть все записи из таблицы covid_data и из её пересечения с vaccinations_data,но не более.
Результат объединения занесите в переменную covid_df.
Сохраните таблицу covid_df — она понадобится нам в следующем юните.

Сколько строк и столбцов в таблице covid_df?
Введите ответ в виде двух чисел через дефис (например, 333-33): первое число — количество строк, второе число — количество столбцов."""
covid_df_0 = covid_data[(covid_data['date'] >= time_start) & (covid_data['date'] <= time_finish)].merge(vaccinations_data, on=['date', 'country'], how='left')
covid_df_0.sort_values(by='date')

covid_df = covid_data.merge(
    vaccinations_data,
    on = ['date', 'country'] ,
    how = 'left') 

covid_df.to_csv('data/covid_df.csv', index=False)
display(covid_df)

ValueError: You are trying to merge on datetime64[ns] and object columns for key 'date'. If you wish to proceed you should use pd.concat

In [ ]:
"""В получившейся в задании 3.3 
таблице covid_df создайте признаки death_rate — общий процент смертей среди зафиксированных случаев (летальность) и 
recover_rate — общий процент случаев выздоровления. Данные характеристики рассчитайте как отношение числа смертей (deaths) и числа выздоровлений (recovered)
к числу зафиксированных случаев (confirmed) и умножьте результаты на 100%."""
covid_df["death_rate"] = covid_df["deaths"] / covid_df["confirmed"] *100
covid_df["recover_rate"] = covid_df["recovered"] / covid_df["confirmed"] *100

In [ ]:
"""Задание 3.4 Какова максимальная летальность в США (United States) за весь период? Ответ округлите до второго знака после запятой."""
daily_deaths_max_us = covid_df[covid_df['country'] == 'United States']['death_rate'].max()
print(daily_deaths_max_us)

7.18954248366013


In [ ]:
"""Задание 3.5 Чему равен средний процент выздоровевших в России (Russia)? Ответ округлите до второго знака после запятой."""
daily_recovered_rf_mean = covid_df[covid_df['country'] == 'Russia']['recover_rate'].mean()
print(daily_recovered_rf_mean)

67.06273489477655
